In [2]:
from vec_db import VecDB
import numpy as np 

## **For each size a folder named "saved_db" is generated , zip this folder and upload to drive**

## Generate Indices for : 1M , 5M , 10M 

In [3]:
rng = np.random.default_rng(200)
# 10**6 , 5*10**6 , 10**7
vectors = rng.random((15*10**6,70) , dtype=np.float32)

# i = 0
# records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(vectors)]

In [4]:
db = VecDB()
db.insert_records(vectors) #=> it will take time 

before mp
begin clustering
Init 1/1 with method k-means++
Inertia for init 1/1: 1773617.75
[MiniBatchKMeans] Reassigning 1055 cluster centers.
Minibatch step 1/15000: mean batch inertia: 5.910730625
Minibatch step 2/15000: mean batch inertia: 4.7626521875, ewa inertia: 4.7626521875
Minibatch step 3/15000: mean batch inertia: 4.7326240625, ewa inertia: 4.762251812526691
Minibatch step 4/15000: mean batch inertia: 4.7146278125, ewa inertia: 4.761616825902002
[MiniBatchKMeans] Reassigning 1148 cluster centers.
Minibatch step 5/15000: mean batch inertia: 4.7042459375, ewa inertia: 4.760851880774305
Minibatch step 6/15000: mean batch inertia: 4.69928625, ewa inertia: 4.7600310057520385
Minibatch step 7/15000: mean batch inertia: 4.69417, ewa inertia: 4.759152859067222
Minibatch step 8/15000: mean batch inertia: 4.6874709375, ewa inertia: 4.75819710017671
Minibatch step 9/15000: mean batch inertia: 4.6838040625, ewa inertia: 4.757205193073814
Minibatch step 10/15000: mean batch inertia: 4.68

KeyboardInterrupt: 

In [ ]:
query_seed = 100 
rng_query = np.random.default_rng(query_seed)

query = rng_query.random((1,70) , dtype=np.float32)   

## Test Clusters (50 , 100 , 150) --> I've been using 50 in vec_db and it's fine 
## **Test Before Uploading** 

In [ ]:
import time
from dataclasses import dataclass
from typing import List

AVG_OVERX_ROWS = 10


@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]


def run_queries(db, np_rows, top_k, num_runs):
    results = []
    for _ in range(num_runs):
        query = np.random.random((1, 70))

        tic = time.time()
        db_ids = db.retrive(query, top_k)
        toc = time.time()
        run_time = toc - tic

        tic = time.time()
        actual_ids = (
            np.argsort(
                np_rows.dot(query.T).T
                / (np.linalg.norm(np_rows, axis=1) * np.linalg.norm(query)),
                axis=1,
            )
            .squeeze()
            .tolist()[::-1]
        )
        toc = time.time()
        np_run_time = toc - tic

        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results


def eval(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append(-1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

In [ ]:
res = run_queries(db, vectors, 5, 10)
print(eval(res))

In [ ]:
del db
del vectors